## Lab11 - DAPP

### Setup

1. Clone the "skeleton" structure from the github repository `https://git@github.com/RoyLai-InfoCorp/fin579`.

    DAPP development involves a large part of HTML and CSS apart from Javascript. To focus on the Smart Contract interaction logic, we will use a pre-built Javascript React project which you are downloading from github for this course.

    ```sh
    $ git clone https://git@github.com/RoyLai-InfoCorp/fin579
    ```

2. Create `lab-11` directory.

    ```sh
    $ md lab-11;
    ```

3. Copy the `fin579-dapp-skeleton` directory from the `fin579` repository into the `lab-11` directory and rename it as `dapp`. The directory structure should look like this:

    ```sh
    $ mv fin579/fin579-dapp-skeleton lab-11/dapp
    $ cd lab-11/dapp
    ```

    /lab-11
    /dapp
    /public
    /src
    package.json

4. Change into the `dapp` directory and install the node packages.

    ```sh
    $ npm i
    ```

5. Start the application and open your browser at http://localhost:3000

    ```sh
    $ npm start
    ```

    The page should be started and looks like this:

    ![fin579-dapp-ui](./img/fin579-dapp-ui.png)

6. Stop the server by pressing CTRL+C in the terminal.

7. Open the file `App.js` in the `\dapp\src` directory.

    This contains the UI logic for the app and pay attention to the section of the code that is importing functions from the `dapp.js` file.

    ```js
    ...
    import {
        getAccount,
        getBalance,
        sellTokens,
        buyTokens,
        getAddressA,
        getAddressB,
    } from "./dapp";
    ```

    There is no need to make changes to the `App.js` file or understand the UI logic. The UI logic will make use of the 6 functions in the `dapp.js` to interact with the uniswap contracts.

8. Open the file `dapp.js` which contains the following code.

    ```js
    const DAI_ADDRESS = "";
    const TUT_ADDRESS = ""; // From Lab 6

    const getAccount = async () => {};

    const getAddressA = () => DAI_ADDRESS;

    const getAddressB = () => TUT_ADDRESS;

    const getBalance = async (tokenAddrA, tokenAddrB, account) => {};

    const sellTokens = async (inputAmt, inputAddr, outputAddr, account) => {};

    const buyTokens = async (outputAmt, outputAddr, inputAddr, account) => {
        throw new Error("ECA");
    };

    export {
        getAddressA,
        getAddressB,
        getBalance,
        getAccount,
        sellTokens,
        buyTokens,
    };
    ```

    In this lab, we will focus on implementing the functions to allow the app to get the token balance of DAI, TUT and the pool token. And implement the `sellTokens` function to trade between DAI and TUT. The remaining `buyToken` function will be implemented on your own as part of your ECA assignment.


## Lab 11a - Connecting to Metamask

1.  Install ethers.js

    ```sh
    npm i ethers
    ```

2.  Start the dapp

    ```sh
    npm start
    ```

3.  If you click on the `CHECK` button in the application, it will show an alert `Invalid account`. Our first task is to create the logic to connect to Metamask.

4.  Open dapp.js. Import ethers.js and create the constants for the contract addresses. You can copy these addresses from Lab 10.

    ```js
    import { ethers, BigNumber } from "ethers";
    const UNISWAP_ROUTER_ADDRESS = "0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D";
    const UNISWAP_FACTORY_ADDRESS =
        "0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f";
    const DAI_ADDRESS = "0x6A9865aDE2B6207dAAC49f8bCba9705dEB0B0e6D";
    const TUT_ADDRESS = "0xf4da4f86D35CFA69f56c5cc7f357Ad03634d7a0F"; // From Lab 6
    ```

5.  Create the metamask provider

    ```js
    const provider = new ethers.providers.Web3Provider(window.ethereum);
    ```

6.  Implement the `getAccount` function.

    ```js
    const getAccount = async () => {
        await provider.send("eth_requestAccounts", []); // Login to metamask
        const account = provider.getSigner();
        return account;
    };
    ```

    When this function is called, it will check if the Metamask wallet is unlocked. If not, Metamask will show a popup to ask for your login password. If you are successfully logged in, the function will return the signer object.

7.  Save the `dapp.js` file. The React app will watch for file changes and hot reload automatically. Otherwise, you can manually refresh the browser to see the changes.

8.  Click on the `CHECK` button and it should no longer show an alert.

9.  You should also see the addresses for TokenA and TokenB populated with the DAI and TUT address respectively.


## Lab 11b - Display Balance

In this session, we will implement the logic for `getBalance()` function. If you look in `App.js` file, you will find the `getBalance()` function is called by the handleCheckBalance function when the `CHECK` button is clicked. The result is stored in the state called `balance` which is used by the UI for the properties: `balanceA`, `balanceB`, `liquidity`, `reservesA` and `reservesB`.

1. Implement the getBalance() function in dapp.js.

    ```js
    const getBalance = async (tokenAddrA, tokenAddrB, account) => {
        const address = await account.getAddress();

        const tokenA = new ethers.Contract(
            tokenAddrA,
            ["function balanceOf(address) view returns(uint)"],
            account
        );

        const tokenB = new ethers.Contract(
            tokenAddrB,
            ["function balanceOf(address) view returns(uint)"],
            account
        );

        // Pool
        const factory = new ethers.Contract(
            UNISWAP_FACTORY_ADDRESS,
            ["function getPair(address,address) view returns(address)"],
            account
        );
        const poolAddress = await factory.getPair(tokenAddrA, tokenAddrB);
        const pool = new ethers.Contract(
            poolAddress,
            [
                "function getReserves() view returns(uint112 reserve0,uint112 reserve1,uint32)",
                "function balanceOf(address) view returns(uint)",
            ],
            account
        );

        // Get Reserves
        const { reserve0, reserve1 } = await pool.getReserves();
        const reservesA = tokenAddrA < tokenAddrB ? reserve0 : reserve1;
        const reservesB = tokenAddrA > tokenAddrB ? reserve0 : reserve1;

        return {
            balanceA: (await tokenA.balanceOf(address))?.toString(),
            balanceB: (await tokenB.balanceOf(address))?.toString(),
            liquidity: (await pool.balanceOf(address))?.toString(),
            reservesA: reservesA.toString(),
            reservesB: reservesB.toString(),
        };
    };
    ```

2. Go to the dapp and click on the `CHECK` button.

3. This time the app will show the actual values in Token Balance for TokenA, TokenB and Liquidity as well as the Pool Reserves for TokenA and TokenB

## Lab 11c - Sell Token

When you the run the dapp, you will find the section below the horizontal line is called Token Swap. It contains the functions for selling or buying an exact amount of TokenA or TokenB.

In this session, we will implement the logic for `sellTokens()` function.

If you look in `App.js` file, you will find the `sellTokens()` function is called by the handleSellA and handleSellB function when the `SELL` button is clicked.

-   For example, to sell 2000 DAI tokens, enter 2000 into the textbox for TokenA Amount and click on the sell button. Exactly 2000 DAI tokens will be deducted from your wallet and the same amount will go into the pool's reserve. You will also receive certain amount of TUT in return based on the current price in the pool.

1. Implement the logic for sellToken function.

    ```js
    const GetAmountOut = (amountIn, reserveIn, reserveOut) => {
        amountIn = BigNumber.from(amountIn);
        const amountInWithFee = amountIn.mul(997);
        const numerator = amountInWithFee.mul(reserveOut);
        const denominator = reserveIn.mul(1000).add(amountInWithFee);
        const amountOut = numerator / denominator;
        return BigNumber.from(Math.floor(amountOut));
    };

    const sellTokens = async (inputAmt, inputAddr, outputAddr, account) => {
        // Get Reserves
        const factory = new ethers.Contract(
            UNISWAP_FACTORY_ADDRESS,
            ["function getPair(address,address) view returns(address)"],
            account
        );
        const poolAddress = await factory.getPair(inputAddr, outputAddr);
        const pool = new ethers.Contract(
            poolAddress,
            [
                "function getReserves() view returns(uint112 reserve0,uint112 reserve1,uint32)",
            ],
            account
        );
        const { reserve0, reserve1 } = await pool.getReserves();
        const inputReserves = inputAddr < outputAddr ? reserve0 : reserve1;
        const outputReserves = inputAddr > outputAddr ? reserve0 : reserve1;

        // Get OutputAmt
        const outputAmt = GetAmountOut(
            BigNumber.from(inputAmt),
            BigNumber.from(inputReserves),
            BigNumber.from(outputReserves)
        );

        // Load contract A and contract B
        const uniswap = new ethers.Contract(
            UNISWAP_ROUTER_ADDRESS,
            [
                `function swapExactTokensForTokens(uint,uint,address[],address,uint)`,
            ],
            account
        );

        // Approve router to withdraw 2000 TokenA from trader account
        const inputToken = new ethers.Contract(
            inputAddr,
            ["function approve(address,uint)"],
            account
        );
        const response = await inputToken.approve(
            UNISWAP_ROUTER_ADDRESS,
            inputAmt
        );
        await response.wait();
        console.log("trade: approved. receipt=", response.hash);

        // Trade 2000 TokenA for 1662 TokenB using trader account
        const ts = (await provider.getBlock()).timestamp + 1000;
        await uniswap.swapExactTokensForTokens(
            inputAmt,
            outputAmt,
            [inputAddr, outputAddr],
            await account.getAddress(),
            ts
        );

        return outputAmt;
    };
    ```

2. Go to the dapp and sell 2000 units of DAI. The script is actually sending 2 transactions to the blockchain. The first transaction is the `inputToken.approve()` transaction and the second transaction is the `uniswap.swapExactTokensForTokens()`. That is why Metamask will show you 2 popups asking you to confirm the transaction (and the deduction of ETH for gas). This may take a few seconds to run depending on the blockchain's current load.

3. Once the transaction is completed, click on the `CHECK` button to check your balance again to confirm the amount being transferred between the token balances and the reserves.
